In [1]:
# Instale o Unsloth (suporta Llama, Mistral, etc. com otimizações)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Bibliotecas de Machine Learning e LangChain
!pip install -U "transformers==4.41.2" "datasets[audio]" "accelerate" "bitsandbytes" "peft"
!pip install -U langchain langgraph langchain_core langchain_community langchain_openai

# Bibliotecas de visualização
!pip install matplotlib seaborn plotly kaleido

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-90uneu7y/unsloth_2996833c4f684c72850455a1255ddfad
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-90uneu7y/unsloth_2996833c4f684c72850455a1255ddfad
  Resolved https://github.com/unslothai/unsloth.git to commit cc29dc5c35002546009fa2d488a3f726c04922b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
ERROR: Package 'unsloth' requires a different Python: 3.13.2 not in '<3.13,>=3.9'
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━

In [2]:
!pip install langchain langgraph langchain-community langchain-ollama pandas plotly seaborn matplotlib

In [ ]:
# cniongolo/biomistral

/bin/sh: linha 1: ollama: comando não encontrado


In [1]:
# --- 0. IMPORTAÇÃO DE BIBLIOTECAS ---
import json
import base64
from io import BytesIO
from typing import TypedDict, List, Dict

# Libs de Visualização
import pandas as pd
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt

# Libs do LangChain e LangGraph
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, END

print("Dependências importadas com sucesso.")

# --- 1. CONFIGURAÇÃO DO MODELO E PROMPTS ---

# Defina o modelo Ollama a ser usado. Garanta que o Ollama esteja em execução.
OLLAMA_MODEL = "cniongolo/biomistral"
try:
    llm = Ollama(model=OLLAMA_MODEL, temperature=0.1)
    # Testa a conexão com o Ollama
    llm.invoke("Responda com 'OK' se estiver funcionando.")
    print(f"Conectado com sucesso ao modelo '{OLLAMA_MODEL}' via Ollama.")
except Exception as e:
    print(f"ERRO: Não foi possível conectar ao Ollama. Verifique se ele está em execução e se o modelo '{OLLAMA_MODEL}' foi baixado (`ollama run {OLLAMA_MODEL}`).")
    print(f"Detalhe do erro: {e}")
    exit()


# Este é o "cérebro" do agente. Um prompt detalhado que guia o LLM.
MASTER_PROMPT_TEMPLATE = """Você é um assistente de IA especialista em medicina, treinado para auxiliar profissionais de saúde.
Sua tarefa é analisar um conjunto de dados clínicos em formato JSON e gerar um resumo narrativo claro, estruturado e objetivo para um relatório clínico.

**REGRAS IMPORTANTES:**
1.  **Objetividade:** Analise apenas os dados fornecidos. Não invente informações.
2.  **Terminologia:** Use terminologia médica apropriada, mas de forma clara.
3.  **Segurança:** NUNCA forneça um diagnóstico definitivo. Aponte achados anormais usando frases como "sugere", "indica", "requer atenção".
4.  **Estrutura:** Formate a saída em Markdown com os seguintes títulos: "Resumo Geral", "Sinais Vitais", e "Resultados Laboratoriais".

**DADOS DE ENTRADA (JSON):**
```json
{clinical_data}
````"""

# --- 2. DEFINIÇÃO DAS FERRAMENTAS DE VISUALIZAÇÃO ---

def create_interactive_line_chart(data: dict, title: str) -> str:
    """Cria um gráfico de linhas interativo com Plotly e retorna como HTML."""
    try:
        df = pd.DataFrame(data)
        x_axis = df.columns[0]
        y_axes = df.columns[1:]
        fig = px.line(df, x=x_axis, y=y_axes, title=title, markers=True, template="plotly_white")
        fig.update_layout(legend_title_text='Métricas')
        return pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de linhas: {e}</i></p>"

def create_static_bar_chart(data: dict, title: str) -> str:
    """Cria um gráfico de barras estático com Seaborn e retorna como uma tag <img> em base64."""
    try:
        df = pd.DataFrame(list(data.items()), columns=['Métrica', 'Valor'])
        plt.figure(figsize=(8, 5))
        sns.barplot(data=df, x='Métrica', y='Valor')
        plt.title(title)
        plt.ylabel("Valor")
        plt.xticks(rotation=15, ha='right')
        buf = BytesIO()
        plt.savefig(buf, format="png", bbox_inches='tight')
        plt.close()
        data_b64 = base64.b64encode(buf.getbuffer()).decode("ascii")
        return f'<img src="data:image/png;base64,{data_b64}" alt="{title}"/>'
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de barras: {e}</i></p>"


# --- 3. DEFINIÇÃO DO AGENTE COM LANGGRAPH ---

class AgentState(TypedDict):
    clinical_data: Dict
    summary_text: str
    visualization_plan: List[Dict]
    visualizations: List[str]
    final_report: str

def analysis_node(state: AgentState):
    print(">>> (Nó 1) Analisando dados e gerando resumo...")
    prompt = MASTER_PROMPT_TEMPLATE.format(clinical_data=json.dumps(state['clinical_data']))
    summary = llm.invoke(prompt)
    return {"summary_text": summary.strip()}

def visualization_planning_node(state: AgentState):
    print(">>> (Nó 2) Planejando visualizações...")
    prompt = f"""Com base nos seguintes dados clínicos, decida quais gráficos gerar.
Dados: {json.dumps(state['clinical_data'])}
Responda APENAS com uma lista de JSON, uma para cada gráfico.
Tipos de gráficos disponíveis: 'interactive_line', 'static_bar'.
Cada JSON deve ter: "chart_type", "title" e "data_subset"."""
    response = llm.invoke(prompt)
    try:
        # Tenta carregar a resposta como JSON
        plan = json.loads(response.strip())
    except json.JSONDecodeError:
        print("AVISO: LLM não retornou um JSON válido para o plano de visualização. Nenhum gráfico será gerado.")
        plan = []
    return {"visualization_plan": plan}

def visualization_execution_node(state: AgentState):
    print(">>> (Nó 3) Gerando os gráficos planejados...")
    visualizations = []
    for plan_item in state.get("visualization_plan", []):
        chart_type = plan_item.get("chart_type")
        data_subset = plan_item.get("data_subset")
        title = plan_item.get("title")
        if not all([chart_type, data_subset, title]): continue
        if chart_type == 'interactive_line':
            visualizations.append(create_interactive_line_chart(data_subset, title))
        elif chart_type == 'static_bar':
            visualizations.append(create_static_bar_chart(data_subset, title))
    return {"visualizations": visualizations}

def report_compilation_node(state: AgentState):
    print(">>> (Nó 4) Compilando o relatório final...")
    # Converte o texto markdown do resumo para HTML
    summary_html = "<br>".join(state['summary_text'].splitlines())
    # Junta as visualizações
    visualizations_html = "".join(state['visualizations'])

    report_html = f"""<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Relatório Clínico</title>
    <style>
        body {{ font-family: sans-serif; line-height: 1.6; margin: 0 auto; max-width: 900px; padding: 20px; }}
        h1, h2 {{ color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;}}
        .report-section {{ margin-bottom: 30px; padding: 20px; background-color: #f8f9f9; border: 1px solid #d5dbdb; border-radius: 8px; }}
        img {{ max-width: 100%; height: auto; display: block; margin: 20px auto; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
        .plotly-graph-div {{ margin: 20px auto; }}
    </style>
</head>
<body>
    <h1>Relatório Clínico Gerado por IA</h1>
    <div class="report-section">
        <h2>Análise Narrativa</h2>
        <p>{summary_html}</p>
    </div>
    <div class="report-section">
        <h2>Visualizações de Dados</h2>
        {visualizations_html}
    </div>
</body>
</html>"""
    return {"final_report": report_html}

# Construção do Grafo
workflow = StateGraph(AgentState)
workflow.add_node("analise", analysis_node)
workflow.add_node("planejamento_viz", visualization_planning_node)
workflow.add_node("execucao_viz", visualization_execution_node)
workflow.add_node("compilacao", report_compilation_node)

# Definição das Arestas
workflow.set_entry_point("analise")
workflow.add_edge("analise", "planejamento_viz")
workflow.add_edge("planejamento_viz", "execucao_viz")
workflow.add_edge("execucao_viz", "compilacao")
workflow.add_edge("compilacao", END)

# Compilação do App
app = workflow.compile()
print("Grafo LangGraph compilado com sucesso.")

# --- 4. FUNÇÃO PRINCIPAL E EXECUÇÃO ---

def generate_clinical_report(patient_data: dict) -> str:
    """
    Função principal que encapsula o agente LangGraph.
    """
    initial_state = {"clinical_data": patient_data}
    final_state = app.invoke(initial_state)
    return final_state.get("final_report", "<h1>Erro: Não foi possível gerar o relatório.</h1>")

if __name__ == "__main__":

    # Dados de exemplo para o paciente
    dados_paciente_exemplo = {
        "paciente_id": "P-78901", "idade": 68, "sexo": "F",
        "historico": "Hipertensão Arterial Sistêmica, Diabetes Mellitus tipo 2",
        "sinais_vitais": {
            "hora": [0, 8, 16, 24], "fc_bpm": [88, 95, 92, 86],
            "pa_sistolica_mmhg": [155, 162, 158, 150], "temperatura_c": [36.8, 37.1, 37.0, 36.7]
        },
        "labs": {
            "hemoglobina_g_dl": 11.5, "leucocitos_mm3": 12500, "plaquetas_mm3": 250000,
            "glicemia_jejum_mg_dl": 180, "creatinina_mg_dl": 1.4
        }
    }

    print("\n" + "="*50 + "\nIniciando a geração do relatório...\n" + "="*50)

    # Gera o relatório
    report_html_output = generate_clinical_report(dados_paciente_exemplo)

    # Salva o relatório final em um arquivo HTML
    output_filename = "relatorio_clinico_final.html"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(report_html_output)

    print("\n" + "="*50 + f"\nRelatório gerado! Abra o arquivo '{output_filename}' no seu navegador.\n" + "="*50)

Dependências importadas com sucesso.


/tmp/ipykernel_1920/1194771034.py:25: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=OLLAMA_MODEL, temperature=0.1)


Conectado com sucesso ao modelo 'cniongolo/biomistral' via Ollama.
Grafo LangGraph compilado com sucesso.

Iniciando a geração do relatório...
>>> (Nó 1) Analisando dados e gerando resumo...


KeyboardInterrupt: 

In [3]:
# Arquivo: agente_relatorios.py (Versão Final Corrigida)
# Descrição: Agente de IA para geração de relatórios clínicos com LangGraph e Ollama.
#
import json
import base64
from io import BytesIO
from typing import TypedDict, List, Dict

# Libs de Visualização
import pandas as pd
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt

# Libs do LangChain e LangGraph
# AVISO DE DEPRECIAÇÃO: A classe Ollama foi movida. Para remover o aviso,
# instale a nova biblioteca com `pip install langchain-ollama`
# e mude a importação para: `from langchain_ollama import OllamaLLM as Ollama`
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, END

print("Dependências importadas com sucesso.")

# --- 1. CONFIGURAÇÃO DO MODELO E PROMPTS ---

# Defina o modelo Ollama a ser usado. Garanta que o Ollama esteja em execução.
# O desempenho pode variar entre modelos. Se este não funcionar, 'pubmed-mistral' é uma alternativa.
#OLLAMA_MODEL = "cniongolo/biomistral"
#OLLAMA_MODEL = "phi3"
OLLAMA_MODEL = "medllama2"
try:
    llm = Ollama(model=OLLAMA_MODEL, temperature=0.1)
    # Testa a conexão com o Ollama
    llm.invoke("Responda com 'OK' se estiver funcionando.")
    print(f"Conectado com sucesso ao modelo '{OLLAMA_MODEL}' via Ollama.")
except Exception as e:
    print(f"ERRO: Não foi possível conectar ao Ollama. Verifique se ele está em execução e se o modelo '{OLLAMA_MODEL}' foi baixado (`ollama run {OLLAMA_MODEL}`).")
    print(f"Detalhe do erro: {e}")
    exit()


# PROMPT REFORÇADO (Nó 1): Instruções mais diretas para o resumo narrativo.
MASTER_PROMPT_TEMPLATE = """ATENÇÃO: Sua resposta deve ser exclusivamente em Português do Brasil.

Você é um assistente de IA especialista em medicina. Sua única tarefa é gerar um resumo narrativo a partir dos dados clínicos em JSON abaixo. Siga estritamente a estrutura de formatação com os títulos "Resumo Geral", "Sinais Vitais", e "Resultados Laboratoriais".

**DADOS CLÍNICOS:**
```json
{clinical_data}
```

**RELATÓRIO GERADO:**
"""

# --- 2. DEFINIÇÃO DAS FERRAMENTAS DE VISUALIZAÇÃO ---

def create_interactive_line_chart(data: dict, title: str) -> str:
    """Cria um gráfico de linhas interativo com Plotly e retorna como HTML."""
    try:
        df = pd.DataFrame(data)
        x_axis = df.columns[0]
        y_axes = df.columns[1:]
        fig = px.line(df, x=x_axis, y=y_axes, title=title, markers=True, template="plotly_white")
        fig.update_layout(legend_title_text='Métricas')
        return pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de linhas: {e}</i></p>"

def create_static_bar_chart(data: dict, title: str) -> str:
    """Cria um gráfico de barras estático com Seaborn e retorna como uma tag <img> em base64."""
    try:
        df = pd.DataFrame(list(data.items()), columns=['Métrica', 'Valor'])
        plt.figure(figsize=(8, 5))
        sns.barplot(data=df, x='Métrica', y='Valor')
        plt.title(title)
        plt.ylabel("Valor")
        plt.xticks(rotation=15, ha='right')
        buf = BytesIO()
        plt.savefig(buf, format="png", bbox_inches='tight')
        plt.close()
        data_b64 = base64.b64encode(buf.getbuffer()).decode("ascii")
        return f'<img src="data:image/png;base64,{data_b64}" alt="{title}"/>'
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de barras: {e}</i></p>"


# --- 3. DEFINIÇÃO DO AGENTE COM LANGGRAPH ---

class AgentState(TypedDict):
    clinical_data: Dict
    summary_text: str
    visualization_plan: List[Dict]
    visualizations: List[str]
    final_report: str

def analysis_node(state: AgentState):
    """Nó 1: Analisa os dados e gera o resumo em texto com o LLM."""
    print(">>> (Nó 1) Analisando dados e gerando resumo...")
    prompt = MASTER_PROMPT_TEMPLATE.format(clinical_data=json.dumps(state['clinical_data']))
    summary = llm.invoke(prompt)
    return {"summary_text": summary.strip()}

def visualization_planning_node(state: AgentState):
    """Nó 2: Usa o LLM para planejar quais visualizações criar (com prompt robustecido)."""
    print(">>> (Nó 2) Planejando visualizações...")

    # PROMPT REFORÇADO (Nó 2): Instruções mais imperativas para a geração de JSON.
    prompt = f"""Você é um robô gerador de JSON. Sua única tarefa é criar um plano de visualização com base nos dados abaixo.

**REGRAS CRÍTICAS:**
1.  Sua resposta deve ser **APENAS** o código JSON.
2.  Não escreva nenhuma explicação, saudação ou qualquer texto fora do JSON.
3.  Sua resposta deve começar com `[` e terminar com `]`.

---
**EXEMPLO DE RESPOSTA PERFEITA:**
```json
[
    {{
        "chart_type": "interactive_line",
        "title": "Evolução dos Sinais Vitais",
        "data_subset": {{
            "hora": [0, 8, 16],
            "fc_bpm": [80, 85, 82]
        }}
    }},
    {{
        "chart_type": "static_bar",
        "title": "Resultados Laboratoriais",
        "data_subset": {{
            "hemoglobina_g_dl": 12.1,
            "leucocitos_mm3": 9800
        }}
    }}
]
```
---

**TAREFA REAL:**

## DADOS PARA ANÁLISE:
```json
{json.dumps(state['clinical_data'])}
```

## GERE O JSON AGORA:
"""
    response = llm.invoke(prompt)

    # Limpa a resposta para garantir que estamos tentando decodificar apenas o JSON
    clean_response = response.strip()
    if "```json" in clean_response:
        clean_response = clean_response.split("```json")[1].split("```")[0]
    
    if not clean_response:
        print("AVISO: LLM retornou uma resposta vazia para o plano de visualização.")
        plan = []
    else:
        try:
            plan = json.loads(clean_response)
        except json.JSONDecodeError as e:
            print(f"AVISO: LLM ainda não retornou um JSON válido. Erro: {e}")
            print(f"Resposta recebida do LLM:\\n---\\n{response}\\n---")
            plan = []

    return {"visualization_plan": plan}


def visualization_execution_node(state: AgentState):
    """Nó 3: Executa o plano e cria os gráficos usando as ferramentas."""
    print(">>> (Nó 3) Gerando os gráficos planejados...")
    visualizations = []
    plan = state.get("visualization_plan", [])
    if not plan:
        print("Nenhum plano de visualização encontrado para executar.")
        return {"visualizations": []}
        
    for plan_item in plan:
        chart_type = plan_item.get("chart_type")
        data_subset = plan_item.get("data_subset")
        title = plan_item.get("title")
        if not all([chart_type, data_subset, title]): continue
        if chart_type == 'interactive_line':
            visualizations.append(create_interactive_line_chart(data_subset, title))
        elif chart_type == 'static_bar':
            visualizations.append(create_static_bar_chart(data_subset, title))
    return {"visualizations": visualizations}

def report_compilation_node(state: AgentState):
    """Nó 4: Compila o texto e as visualizações em um relatório HTML final."""
    print(">>> (Nó 4) Compilando o relatório final...")
    # Converte o texto markdown do resumo para HTML
    summary_html = "<br>".join(state['summary_text'].splitlines())
    # Junta as visualizações
    visualizations_html = "".join(state['visualizations'])

    report_html = f"""<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Relatório Clínico</title>
    <style>
        body {{ font-family: sans-serif; line-height: 1.6; margin: 0 auto; max-width: 900px; padding: 20px; }}
        h1, h2 {{ color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;}}
        .report-section {{ margin-bottom: 30px; padding: 20px; background-color: #f8f9f9; border: 1px solid #d5dbdb; border-radius: 8px; }}
        img {{ max-width: 100%; height: auto; display: block; margin: 20px auto; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
        .plotly-graph-div {{ margin: 20px auto; }}
    </style>
</head>
<body>
    <h1>Relatório Clínico Gerado por IA</h1>
    <div class="report-section">
        <h2>Análise Narrativa</h2>
        <p>{summary_html}</p>
    </div>
    <div class="report-section">
        <h2>Visualizações de Dados</h2>
        {visualizations_html if visualizations_html else "<p><i>Nenhuma visualização foi gerada.</i></p>"}
    </div>
</body>
</html>"""
    return {"final_report": report_html}

# Construção do Grafo
workflow = StateGraph(AgentState)
workflow.add_node("analise", analysis_node)
workflow.add_node("planejamento_viz", visualization_planning_node)
workflow.add_node("execucao_viz", visualization_execution_node)
workflow.add_node("compilacao", report_compilation_node)

# Definição das Arestas
workflow.set_entry_point("analise")
workflow.add_edge("analise", "planejamento_viz")
workflow.add_edge("planejamento_viz", "execucao_viz")
workflow.add_edge("execucao_viz", "compilacao")
workflow.add_edge("compilacao", END)

# Compilação do App
app = workflow.compile()
print("Grafo LangGraph compilado com sucesso.")

# --- 4. FUNÇÃO PRINCIPAL E EXECUÇÃO ---

def generate_clinical_report(patient_data: dict) -> str:
    """
    Função principal que encapsula o agente LangGraph.
    """
    initial_state = {"clinical_data": patient_data}
    final_state = app.invoke(initial_state)
    return final_state.get("final_report", "<h1>Erro: Não foi possível gerar o relatório.</h1>")

if __name__ == "__main__":

    # Dados de exemplo para o paciente
    dados_paciente_exemplo = {
        "paciente_id": "P-78901", "idade": 68, "sexo": "F",
        "historico": "Hipertensão Arterial Sistêmica, Diabetes Mellitus tipo 2",
        "sinais_vitais": {
            "hora": [0, 8, 16, 24], "fc_bpm": [88, 95, 92, 86],
            "pa_sistolica_mmhg": [155, 162, 158, 150], "temperatura_c": [36.8, 37.1, 37.0, 36.7]
        },
        "labs": {
            "hemoglobina_g_dl": 11.5, "leucocitos_mm3": 12500, "plaquetas_mm3": 250000,
            "glicemia_jejum_mg_dl": 180, "creatinina_mg_dl": 1.4
        }
    }

    print("\n" + "="*50 + "\nIniciando a geração do relatório...\n" + "="*50)

    # Gera o relatório
    report_html_output = generate_clinical_report(dados_paciente_exemplo)

    # Salva o relatório final em um arquivo HTML
    output_filename = "relatorio_clinico_final.html"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(report_html_output)

    print("\n" + "="*50 + f"\nRelatório gerado! Abra o arquivo '{output_filename}' no seu navegador.\n" + "="*50)



Dependências importadas com sucesso.
Conectado com sucesso ao modelo 'medllama2' via Ollama.
Grafo LangGraph compilado com sucesso.

Iniciando a geração do relatório...
>>> (Nó 1) Analisando dados e gerando resumo...
>>> (Nó 2) Planejando visualizações...
AVISO: LLM ainda não retornou um JSON válido. Erro: Expecting value: line 1 column 1 (char 0)
Resposta recebida do LLM:\n---\n
Please provide the data for the visualization in a JSON format. The data should include the patient's ID, age, sex, medical history, vital signs (including heart rate), and laboratory results (including hemoglobin A1c).\n---
>>> (Nó 3) Gerando os gráficos planejados...
Nenhum plano de visualização encontrado para executar.
>>> (Nó 4) Compilando o relatório final...

Relatório gerado! Abra o arquivo 'relatorio_clinico_final.html' no seu navegador.


In [1]:

# Arquivo: agente_relatorios.py (Versão com Geração de Gráficos Determinística)
# Descrição: Agente de IA que usa o LLM para texto e lógica Python para gráficos.
#
import json
import base64
from io import BytesIO
from typing import TypedDict, List, Dict

# Libs de Visualização
import pandas as pd
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt

# Libs do LangChain e LangGraph
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, END

print("Dependências importadas com sucesso.")

# --- 1. CONFIGURAÇÃO DO MODELO E PROMPTS ---

OLLAMA_MODEL = "medllama2"
try:
    llm = Ollama(model=OLLAMA_MODEL, temperature=0.1)
    llm.invoke("Responda com 'OK' se estiver funcionando.")
    print(f"Conectado com sucesso ao modelo '{OLLAMA_MODEL}' via Ollama.")
except Exception as e:
    print(f"ERRO: Não foi possível conectar ao Ollama. Verifique se ele está em execução e se o modelo '{OLLAMA_MODEL}' foi baixado (`ollama run {OLLAMA_MODEL}`).")
    exit()

# PROMPT REFORÇADO (Nó 1): Instruções mais diretas para o resumo narrativo.
MASTER_PROMPT_TEMPLATE = """ATENÇÃO: Sua resposta deve ser exclusivamente em Português do Brasil.

Você é um assistente de IA especialista em medicina. Sua única tarefa é gerar um resumo narrativo a partir dos dados clínicos em JSON abaixo. Siga estritamente a estrutura de formatação com os títulos "Resumo Geral", "Sinais Vitais", e "Resultados Laboratoriais".

**DADOS CLÍNICOS:**
```json
{clinical_data}
```

**RELATÓRIO GERADO:**
"""

# --- 2. DEFINIÇÃO DAS FERRAMENTAS DE VISUALIZAÇÃO ---
# (As funções de criar gráficos permanecem as mesmas)

def create_interactive_line_chart(data: dict, title: str) -> str:
    """Cria um gráfico de linhas interativo com Plotly e retorna como HTML."""
    try:
        df = pd.DataFrame(data)
        x_axis = df.columns[0]
        y_axes = df.columns[1:]
        fig = px.line(df, x=x_axis, y=y_axes, title=title, markers=True, template="plotly_white")
        fig.update_layout(legend_title_text='Métricas')
        return pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de linhas: {e}</i></p>"

def create_static_bar_chart(data: dict, title: str) -> str:
    """Cria um gráfico de barras estático com Seaborn e retorna como uma tag <img> em base64."""
    try:
        df = pd.DataFrame(list(data.items()), columns=['Métrica', 'Valor'])
        plt.figure(figsize=(8, 5))
        sns.barplot(data=df, x='Métrica', y='Valor')
        plt.title(title)
        plt.ylabel("Valor")
        plt.xticks(rotation=15, ha='right')
        buf = BytesIO()
        plt.savefig(buf, format="png", bbox_inches='tight')
        plt.close()
        data_b64 = base64.b64encode(buf.getbuffer()).decode("ascii")
        return f'<img src="data:image/png;base64,{data_b64}" alt="{title}"/>'
    except Exception as e:
        return f"<p><i>Erro ao gerar gráfico de barras: {e}</i></p>"


# --- 3. DEFINIÇÃO DO AGENTE COM LANGGRAPH (ARQUITETURA SIMPLIFICADA) ---

class AgentState(TypedDict):
    clinical_data: Dict
    summary_text: str
    visualizations: List[str]  # Não precisamos mais do 'visualization_plan'
    final_report: str

def analysis_node(state: AgentState):
    """Nó 1: Analisa os dados e gera o resumo em texto com o LLM."""
    print(">>> (Nó 1) Analisando dados e gerando resumo...")
    prompt = MASTER_PROMPT_TEMPLATE.format(clinical_data=json.dumps(state['clinical_data']))
    summary = llm.invoke(prompt)
    return {"summary_text": summary.strip()}

def deterministic_visualization_node(state: AgentState):
    """Nó 2: Gera gráficos de forma determinística com base na estrutura dos dados."""
    print(">>> (Nó 2) Gerando gráficos com lógica Python...")
    visualizations = []
    data = state.get("clinical_data", {})

    # REGRA 1: Se houver 'sinais_vitais', crie um gráfico de linhas.
    if "sinais_vitais" in data and isinstance(data["sinais_vitais"], dict):
        print("   - Encontrados dados de 'sinais_vitais'. Gerando gráfico de linhas.")
        sinais_vitais_data = data["sinais_vitais"]
        visualizations.append(create_interactive_line_chart(sinais_vitais_data, "Evolução dos Sinais Vitais"))

    # REGRA 2: Se houver 'labs', crie um gráfico de barras.
    if "labs" in data and isinstance(data["labs"], dict):
        print("   - Encontrados dados de 'labs'. Gerando gráfico de barras.")
        labs_data = data["labs"]
        visualizations.append(create_static_bar_chart(labs_data, "Resultados Laboratoriais"))
    
    if not visualizations:
        print("   - Nenhum dado apropriado para visualização foi encontrado.")

    return {"visualizations": visualizations}

def report_compilation_node(state: AgentState):
    """Nó 3: Compila o texto e as visualizações em um relatório HTML final."""
    print(">>> (Nó 3) Compilando o relatório final...")
    summary_html = "<br>".join(state.get('summary_text', '').splitlines())
    visualizations_html = "".join(state.get('visualizations', []))

    report_html = f"""<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Relatório Clínico</title>
    <style>
        body {{ font-family: sans-serif; line-height: 1.6; margin: 0 auto; max-width: 900px; padding: 20px; }}
        h1, h2 {{ color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;}}
        .report-section {{ margin-bottom: 30px; padding: 20px; background-color: #f8f9f9; border: 1px solid #d5dbdb; border-radius: 8px; }}
        img {{ max-width: 100%; height: auto; display: block; margin: 20px auto; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
        .plotly-graph-div {{ margin: 20px auto; }}
    </style>
</head>
<body>
    <h1>Relatório Clínico Gerado por IA</h1>
    <div class="report-section">
        <h2>Análise Narrativa</h2>
        <p>{summary_html}</p>
    </div>
    <div class="report-section">
        <h2>Visualizações de Dados</h2>
        {visualizations_html if visualizations_html else "<p><i>Nenhuma visualização foi gerada.</i></p>"}
    </div>
</body>
</html>"""
    return {"final_report": report_html}

# Construção do Grafo (mais simples agora)
workflow = StateGraph(AgentState)
workflow.add_node("analise", analysis_node)
workflow.add_node("geracao_determinista_viz", deterministic_visualization_node)
workflow.add_node("compilacao", report_compilation_node)

# Definição das Arestas
workflow.set_entry_point("analise")
workflow.add_edge("analise", "geracao_determinista_viz")
workflow.add_edge("geracao_determinista_viz", "compilacao")
workflow.add_edge("compilacao", END)

# Compilação do App
app = workflow.compile()
print("Grafo LangGraph compilado com sucesso.")

# --- 4. FUNÇÃO PRINCIPAL E EXECUÇÃO ---

def generate_clinical_report(patient_data: dict) -> str:
    """Função principal que encapsula o agente LangGraph."""
    initial_state = {"clinical_data": patient_data}
    final_state = app.invoke(initial_state)
    return final_state.get("final_report", "<h1>Erro: Não foi possível gerar o relatório.</h1>")

if __name__ == "__main__":
    dados_paciente_exemplo = {
        "paciente_id": "P-78901", "idade": 68, "sexo": "F",
        "historico": "Hipertensão Arterial Sistêmica, Diabetes Mellitus tipo 2",
        "sinais_vitais": {
            "hora": [0, 8, 16, 24], "fc_bpm": [88, 95, 92, 86],
            "pa_sistolica_mmhg": [155, 162, 158, 150], "temperatura_c": [36.8, 37.1, 37.0, 36.7]
        },
        "labs": {
            "hemoglobina_g_dl": 11.5, "leucocitos_mm3": 12500, "plaquetas_mm3": 250000,
            "glicemia_jejum_mg_dl": 180, "creatinina_mg_dl": 1.4
        }
    }
    
    print("\n" + "="*50 + "\nIniciando a geração do relatório...\n" + "="*50)
    report_html_output = generate_clinical_report(dados_paciente_exemplo)
    output_filename = "relatorio_clinico_final.html"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(report_html_output)
        
    print("\n" + "="*50 + f"\nRelatório gerado! Abra o arquivo '{output_filename}' no seu navegador.\n" + "="*50)



Dependências importadas com sucesso.


/tmp/ipykernel_456/2927279805.py:26: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=OLLAMA_MODEL, temperature=0.1)


Conectado com sucesso ao modelo 'medllama2' via Ollama.
Grafo LangGraph compilado com sucesso.

Iniciando a geração do relatório...
>>> (Nó 1) Analisando dados e gerando resumo...
>>> (Nó 2) Gerando gráficos com lógica Python...
   - Encontrados dados de 'sinais_vitais'. Gerando gráfico de linhas.
   - Encontrados dados de 'labs'. Gerando gráfico de barras.
>>> (Nó 3) Compilando o relatório final...

Relatório gerado! Abra o arquivo 'relatorio_clinico_final.html' no seu navegador.
